In [2]:
!pip install huggingface-hub
!pip install diffusers
!pip install segment-anything
!pip install lama-cleane

  Obtaining dependency information for diffusers from https://files.pythonhosted.org/packages/74/2b/69bb842f7567cd92a540f8a9a63a20e09304ad8ff84530f26762e7e19626/diffusers-0.30.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB 1.4 MB/s eta 0:00:02
   - -------------------------------------- 0.1/2.6 MB 1.7 MB/s eta 0:00:02
   ------ --------------------------------- 0.4/2.6 MB 3.7 MB/s eta 0:00:01
   ------------ --------------------------- 0.8/2.6 MB 5.1 MB/s eta 0:00:01
   ----------------- ---------------------- 1.1/2.6 MB 5.6 MB/s eta 0:00:01
   ----------------------- ---------------- 1.5/2.6 MB 6.1 MB/s eta 0:00:01
   ---------------------------- ----------- 1.9/2.6 MB 6.4 MB/s eta 0:00:01
   ------------------------------------ --- 2.4/2.6 MB 6.9 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 6.7 MB/s eta 0:00:00
  Obtaining dependency information for segm

ERROR: Could not find a version that satisfies the requirement lama-cleane (from versions: none)
ERROR: No matching distribution found for lama-cleane


In [6]:
import os
import cv2
import numpy as np
from segment_anything import sam_model_registry, SamPredictor
from PIL import Image

def detect_circles(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
                               param1=50, param2=30, minRadius=20, maxRadius=100)
    
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
    return circles

def generate_masks(input_folder, output_folder, sam_checkpoint):
    os.makedirs(output_folder, exist_ok=True)

    # Initialize SAM
    sam = sam_model_registry["default"](checkpoint=sam_checkpoint)
    predictor = SamPredictor(sam)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Detect circles
            circles = detect_circles(image_rgb)

            if circles is not None:
                # Set image for SAM predictor
                predictor.set_image(image_rgb)

                # Create a blank black image for the final mask
                final_mask = np.zeros(image.shape[:2], dtype=np.uint8)

                for (x, y, r) in circles:
                    # Generate points around the circle
                    num_points = 12
                    points = []
                    for i in range(num_points):
                        angle = 2 * np.pi * i / num_points
                        px = int(x + r * np.cos(angle))
                        py = int(y + r * np.sin(angle))
                        points.append([px, py])

                    # Get masks from SAM
                    input_point = np.array(points)
                    input_label = np.array([1] * len(points))  # 1 for foreground
                    masks, _, _ = predictor.predict(
                        point_coords=input_point,
                        point_labels=input_label,
                        multimask_output=False
                    )

                    # Add the mask to the final mask
                    final_mask = np.logical_or(final_mask, masks[0]).astype(np.uint8)

                # Convert boolean mask to uint8 (0 and 255)
                final_mask = (final_mask * 255).astype(np.uint8)

                # Save the mask
                output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_mask.png")
                Image.fromarray(final_mask).save(output_path)

                print(f"Processed {filename}")
            else:
                print(f"No circles detected in {filename}")

    print("All images processed.")

# Usage
input_folder = "synthetic"
output_folder = "synthetic_mask"
sam_checkpoint = "sam_vit_h_4b8939.pth"

generate_masks(input_folder, output_folder, sam_checkpoint)

Processed image_1003 - Copy.png
Processed image_101.png
No circles detected in image_104 - Copy.png
No circles detected in image_105 - Copy.png
Processed image_1062 - Copy.png
No circles detected in image_1098 - Copy.png
No circles detected in image_111.png
No circles detected in image_1126 - Copy.png
Processed image_1167 - Copy.png
Processed image_1180 - Copy.png
Processed image_1180.png
Processed image_1183 - Copy.png
No circles detected in image_119.png
No circles detected in image_1200 - Copy.png
Processed image_1239 - Copy.png
No circles detected in image_124 - Copy.png
Processed image_1266 - Copy.png
No circles detected in image_1279 - Copy.png
Processed image_1288.png
Processed image_1299 - Copy.png
Processed image_1303 - Copy.png
Processed image_1310 - Copy.png
No circles detected in image_1343 - Copy.png
Processed image_135.png
No circles detected in image_136 - Copy.png
No circles detected in image_1361 - Copy.png
Processed image_1369 - Copy.png
Processed image_1369.png
No ci